In [1]:
# Copyright (c) 2017, Gerti Tuzi
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without
# modification, are permitted provided that the following conditions are met:
#     * Redistributions of source code must retain the above copyright
#       notice, this list of conditions and the following disclaimer.
#     * Redistributions in binary form must reproduce the above copyright
#       notice, this list of conditions and the following disclaimer in the
#       documentation and/or other materials provided with the distribution.
#     * Neither the name of Gerti Tuzi nor the
#       names of its contributors may be used to endorse or promote products
#       derived from this software without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE
# DISCLAIMED. IN NO EVENT SHALL <COPYRIGHT HOLDER> BE LIABLE FOR ANY
# DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES
# (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES;
# LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND
# ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT
# (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS
# SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.
#####################################################################################

# Basic Regression Model Training

## Data

#### Load Logfile

In [1]:
import csv
import cv2
import numpy as np
# -------------------------
# Load data from the csv file
loglines = []
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        loglines.append(line)

#### Split Train/Validation/Testing

In [2]:
from sklearn.model_selection import train_test_split
validation_portion = 0.3
# Log lines are our references to the samples
train_loglines, test_loglines = train_test_split(loglines, test_size=0.333, random_state=0)
train_loglines, validation_loglines = train_test_split(train_loglines, test_size=validation_portion, random_state=0)

num_cams = 3
# 3 cameras * (1 original + 1 horizontal flip)
n_train = len(train_loglines) * 2 * num_cams
n_valid = len(validation_loglines)
n_test = len(test_loglines)

print('Num Train Samples: ' + str(n_train))
print('Num Valid Samples: ' + str(n_valid))
print('Num Test Samples: ' + str(n_test))


Num Train Samples: 72912
Num Valid Samples: 5208
Num Test Samples: 8668


## Train Model

In [3]:
from GTRegressionModel import GTRegressionModel
import math, os
from keras.callbacks import ModelCheckpoint, EarlyStopping,LearningRateScheduler
from DataGenerators import train_generator_center, train_generator_3, generator

mode = 'angle'

model_dir = 'model'
if mode == 'angle':
    best_model = model_dir + '/model_angle.h5'
elif mode == 'velocity':
    best_model = model_dir + '/model_velocity.h5'
else:
    raise Exception('Training mode must be specified: Y is what measure ?')
    

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

# Training parameters
Initial_LR = 0.001
Train_batch_size = 90 # multiples of 6
Valid_batch_size = 32
Test_batch_size = 32
Dropout_Rate = 0.2
Max_No_Epochs = 35


Num_Batches_Per_Train_Epoch = int(n_train/Train_batch_size)
if (n_train  % Train_batch_size) > 0:
    Num_Batches_Per_Train_Epoch += 1

    
Num_Batches_Per_Valid_Epoch = int(n_valid/Valid_batch_size)
if (n_valid  % Valid_batch_size) > 0:
    Num_Batches_Per_Valid_Epoch += 1

    
Num_Batches_Per_Test_Epoch = int(n_test/Test_batch_size)
if (n_test  % Test_batch_size) > 0:
    Num_Batches_Per_Test_Epoch += 1
    

Using TensorFlow backend.


### Data Augmentation
3 cameras are placed on the vehicle. Center camera is used for online driving. Left/Right cameras emulate (augment)  horizontal drifts (shifts).

Data is augmented in the following ways:
* Horizontal flip: simple left-right flip of the image, and a sign-change of the target steering angle
* Using Left/Right camera images as shifted center images. An offset is applied to the steering angle (data label) when using these camera images, in the following way: positive steering (counter-clockwise) for the left-camera, negative steering (clockwise) for the right-camera.
* Random artificial horizontal shifts during training: shift the image in proportion to the size of the width. Adjust the target steering angle in proportion of the shift: `<new_angle> = <old_angle> + <shift_proportion> x <old_angle>`
* Random vertical shifts/rotations/shears: random shifts which do not affect the angle label. **We should not apply too much vertical shifts** because we're clipping the top and bottom of the image before feeding it into the NN model.


### Data generators
Using data generators because the dataset is very large

**Train generator** performs the following transformations:
* Horizontal shifts: as a portion of the size of the width of the image. Label target is adjusted according to the propotion of the shift: 
* Vertical shifts: as a proportion of height. Angle size is not changed
* Rotation/Shear rotation: in angle degrees.
* Shuffle the data during training
* Outputs batches of data


**Validation/Test genrators**
* Output only the center camera data. The label (steering angle) is not modified.
* One sample per call.



In [4]:
train_gen = train_generator_3(train_loglines, sixth_of_batch_size=int(Train_batch_size/6.),
                            horizontal_augment=True, horizontal_shift_range=0.05,
                            vertical_augment=True, vertical_shift_range=0.05,
                            rotation_augment=True, rotation_range=5.,
                            shear_augment=True, shear_range=5.,
                            do_shuffle=True, mode = mode)
test_gen = generator(loglines=test_loglines, batch_size=Test_batch_size, mode=mode)
valid_gen = generator(loglines=validation_loglines, batch_size=Valid_batch_size, mode=mode)

#### Callbacks

Callbacks used for learning rate decay, early termination, etc

In [5]:
# learning rate schedule
def step_decay(epoch):
    # Number of batches before learning rate drops by a 'drop_factor'
    batch_count_drop_step = 10000.0
    drop_factor = 0.1

    batch_count = float(epoch*Num_Batches_Per_Train_Epoch)
    pow_factor = (math.pow(drop_factor, math.floor(batch_count/(batch_count_drop_step))))
    lrate = Initial_LR*pow_factor

    print("Epoch: {0} - Batch_count: {1} - Pow Fact: {2:0.3f} - lrate: {3:0.6f}".
          format(epoch, batch_count, pow_factor, lrate))
    return lrate


# learning schedule callback
lrate = LearningRateScheduler(step_decay)


# The best model here is determined by the monitor parameter.
# It can be the best based on training values, or validataion (prefixed by val_)
best_val_mse_checkpoint = ModelCheckpoint(best_model,
                             monitor= 'val_loss', # This determines what gets saved (val_acc: validation accuracy)
                             verbose=1,
                             save_best_only=True,
                             save_weights_only= False, # If false the whole model is saved
                             mode='auto')


earlystopping = EarlyStopping(monitor='val_loss',
                              #  minimum change in the monitored quantity to qualify as an improvement,
                              # i.e. an absolute change of less than min_delta, will count as no improvement.
                              min_delta=0.0,
                              # number of epochs with no improvement after
                              # which training will be stopped
                              patience=30,
                              verbose=1,
                              mode='auto')

callbacks_list = [best_val_mse_checkpoint, earlystopping, lrate]

#### Model

In [6]:
model = GTRegressionModel(input_shape=(160, 320, 3), drop_prob=Dropout_Rate)

# Minimizing the error between the logits and the
# continuous value of the labels (steering wheel angle)
# we get a regression fit (as opposed to classification)
# Using the loss function MSE - the same loss function
# used in regular regression
model.compile(loss='mse', optimizer='adam')


#### Train Model
Train the model with custom generators above

In [7]:
print("************** Parameters **********************")
print (("- Train Batch_Size : {0}\n- Max_No_Epochs: {1}\n" +
        "- Init_LR: {2}\n- Num_Batches_Per_Epoch: {3}\n" +
        "- Dropout_Rate: {4:0.3f}\n"+ 
        "- Training Mode: {5}").format(
    Train_batch_size, Max_No_Epochs,
    Initial_LR,Num_Batches_Per_Train_Epoch,Dropout_Rate, mode))
print("***************************************************")


train_history = model.fit_generator(generator=train_gen, steps_per_epoch= Num_Batches_Per_Train_Epoch,
                    validation_data=valid_gen, validation_steps = Num_Batches_Per_Valid_Epoch,
                    epochs=Max_No_Epochs,
                    callbacks=callbacks_list,
                    verbose = 2)


************** Parameters **********************
- Train Batch_Size : 90
- Max_No_Epochs: 35
- Init_LR: 0.001
- Num_Batches_Per_Epoch: 811
- Dropout_Rate: 0.200
- Training Mode: angle
***************************************************
Epoch: 0 - Batch_count: 0.0 - Pow Fact: 1.000 - lrate: 0.001000
Epoch 1/35
Epoch 00000: val_loss improved from inf to 0.07998, saving model to model/model_angle.h5
1558s - loss: 0.1256 - val_loss: 0.0800
Epoch: 1 - Batch_count: 811.0 - Pow Fact: 1.000 - lrate: 0.001000
Epoch 2/35
Epoch 00001: val_loss improved from 0.07998 to 0.07348, saving model to model/model_angle.h5
1534s - loss: 0.1018 - val_loss: 0.0735
Epoch: 2 - Batch_count: 1622.0 - Pow Fact: 1.000 - lrate: 0.001000
Epoch 3/35
Epoch 00002: val_loss improved from 0.07348 to 0.07075, saving model to model/model_angle.h5
1533s - loss: 0.0960 - val_loss: 0.0708
Epoch: 3 - Batch_count: 2433.0 - Pow Fact: 1.000 - lrate: 0.001000
Epoch 4/35
Epoch 00003: val_loss improved from 0.07075 to 0.06940, savin

#### Save results

In [8]:
if mode == 'angle':
    model.save( model_dir + '/final_model_angle.h5')
elif mode == 'velocity':
    model.save( model_dir + '/final_model_velocity.h5')    

    

# Save history
import pickle
with open('train_history.pickle', 'wb') as handle:
    pickle.dump(train_history.history, handle, protocol=pickle.HIGHEST_PROTOCOL)


### Evaluate on Test Data

In [9]:
# Evaluate on test data
scores = model.evaluate_generator(generator=test_gen, steps=Num_Batches_Per_Test_Epoch)
print("Test MSE: {0:.2f}".format(scores))

print(model.metrics_names)


Test MSE: 0.06
['loss']
